In [1]:
import boto3
from boto3.s3.transfer import S3Transfer
import os
import import_ipynb
# from pdf2Image import convert_pdf
# from create_bucket import create_bucket,upload_file_s3,textract_job

In [2]:
!pip install pdf2Image

## Converting pdf2Images
The below function heps in converting pdf's to Images


In [3]:
from pdf2image import convert_from_path
import os

In [4]:
# Run this command -- ?
conda install -c conda-forge poppler

SyntaxError: invalid syntax (<ipython-input-4-781bfa385e5e>, line 2)

In [ ]:
import os
print(os.getcwd())

In [32]:
# root directory
#conversion
def convert_pdf_Img(root_dir, save_dir, res=400):
    
    for pdf_path in os.listdir(root_dir):
        path = (root_dir +'\\' + pdf_path)
        pages = convert_from_path(path,res)

        name_with_extension = pdf_path.rsplit('/')[-1]
        name = name_with_extension.rsplit('.')[0]

        out_path = os.path.join(save_dir,name)
        os.makedirs(out_path)

        for idx,page in enumerate(pages):
            page.save(f'{out_path}/{name}_{idx}.png','PNG')

## Starting the Textract Job

In [78]:
# decision making whether to go for key-value or the table output
def textract_job(data_dir):
    s3_client = boto3.client('s3')
    tess = boto3.client('textract', region_name='ap-south-1')
    bucket = 'arthashastra-1'
    print(1)
    print("MAKE CHOICE FOR THE INPUT:")
    print("=============================")
    print("KV == KEY_VALUE PAIR \nTABLE == TABLE OUTPUT")
    
    inpt = input('enter the input : ')
    
    if inpt == 'KV':
        output_file = os.path.join(data_dir,'Output_KV.csv')
        path = os.path.join(data_dir,'Test_Inv')
        print(path)
        with open(output_file, "wt") as fout:
            files = os.listdir(path)
            for file in files:
                fle = os.listdir(os.path.join(path,file))
                for f in fle:
                    file_name = os.path.join(path,file,f)
                    print(f)
                    main_KV(file_name,fout)
                    
    elif inpt=="TABLE":
        output_file = os.path.join(data_dir,'Output.csv')
        path = os.path.join(data_dir,'Test_Inv')
        with open(output_file, "wt") as fout:
            files = os.listdir(path)
            for file in files:
                fle = os.listdir(os.path.join(path,file))
                for f in fle:
                    file_name = os.path.join(path,file,f)
                    print(f)
                    main(file_name,fout)
    else:
        print("****=============================****")
        print("please chooce the right option")
        textract_job(data_dir)
    

## Table Function

In [79]:
import webbrowser, os
import json
import boto3
import io
from io import BytesIO
import sys
from pprint import pprint

In [97]:
def get_rows_columns_map(table_result, blocks_map):
    rows = {}
    for relationship in table_result['Relationships']:
        if relationship['Type'] == 'CHILD':
            for child_id in relationship['Ids']:
                cell = blocks_map[child_id]
                if cell['BlockType'] == 'CELL':
                    row_index = cell['RowIndex']
                    col_index = cell['ColumnIndex']
                    if row_index not in rows:
                        # create new row
                        rows[row_index] = {}

                    # get the text value
                    rows[row_index][col_index] = get_text(cell, blocks_map)
    return rows

def get_text(result, blocks_map):
    text = ''
    if 'Relationships' in result:
        for relationship in result['Relationships']:
            if relationship['Type'] == 'CHILD':
                for child_id in relationship['Ids']:
                    word = blocks_map[child_id]
                    if word['BlockType'] == 'WORD':
                        text += word['Text'] + ' '
                    if word['BlockType'] == 'SELECTION_ELEMENT':
                        if word['SelectionStatus'] == 'SELECTED':
                            text += 'X '
    return text

def get_table_csv_results(file_name):
    with open(file_name, 'rb') as file:
        img_test = file.read()
        bytes_test = bytearray(img_test)
        print('Image loaded', file_name)

    # process using image bytes
    # get the results
    client = boto3.client('textract')

    response = client.analyze_document(Document={'Bytes': bytes_test}, FeatureTypes=['TABLES'])

    # Get the text blocks
    blocks = response['Blocks']


    blocks_map = {}
    table_blocks = []
    for block in blocks:
        blocks_map[block['Id']] = block
        if block['BlockType'] == "TABLE":
            table_blocks.append(block)

    if len(table_blocks) <= 0:
        return "<b> NO Table FOUND </b>"

    csv = ''
    for index, table in enumerate(table_blocks):
        csv += generate_table_csv(file_name, table, blocks_map, index + 1)
        csv += '\n\n'

    return csv


def generate_table_csv(file_name, table_result, blocks_map, table_index):
    rows = get_rows_columns_map(table_result, blocks_map)
    
    page_name = file_name.split('\\')[-1]
    table_id = 'Table_' + str(table_index) +'\\'+ page_name
    print(table_id)
    # get cells.
    csv = 'Table: {0}\n\n'.format(table_id)
    

    for row_index, cols in rows.items():

        for col_index, text in cols.items():
            csv += '{}'.format(text) + ","
        csv += '\n'

    csv += '\n\n\n'
    return csv

def main(file_name,fout):
    table_csv = get_table_csv_results(file_name)

    #output_file = 'C:/Users/lenovo/PycharmProjects/aws-textract1/Output.csv'

    # replace content
    #fout.write(file_name)
    fout.write(table_csv)

    # show the results
    #print('CSV OUTPUT FILE: ', output_file)




In [118]:
csv = 'Table: {0}\n\n' \n'k'
csv

SyntaxError: unexpected character after line continuation character (<ipython-input-118-6c0c2b830925>, line 1)

## Key-Value pair Function

In [98]:
import boto3
import sys
import re
import json


In [99]:
def get_kv_map(file_name):

    with open(file_name, 'rb') as file:
        img_test = file.read()
        bytes_test = bytearray(img_test)
        print('Image loaded', file_name)

    # process using image bytes
    client = boto3.client('textract')
    response = client.analyze_document(Document={'Bytes': bytes_test}, FeatureTypes=['FORMS'])

    # Get the text blocks
    blocks=response['Blocks']
    

    # get key and value maps
    key_map = {}
    value_map = {}
    block_map = {}
    for block in blocks:
        block_id = block['Id']
        block_map[block_id] = block
        if block['BlockType'] == "KEY_VALUE_SET":
            if 'KEY' in block['EntityTypes']:
                key_map[block_id] = block
            else:
                value_map[block_id] = block

    return key_map, value_map, block_map


def get_kv_relationship(key_map, value_map, block_map):
    kvs = {}
    for block_id, key_block in key_map.items():
        value_block = find_value_block(key_block, value_map)
        key = get_text(key_block, block_map)
        val = get_text(value_block, block_map)
        kvs[key] = val
    return kvs

def find_value_block(key_block, value_map):
    for relationship in key_block['Relationships']:
        if relationship['Type'] == 'VALUE':
            for value_id in relationship['Ids']:
                value_block = value_map[value_id]
    return value_block


def get_text(result, blocks_map):
    text = ''
    if 'Relationships' in result:
        for relationship in result['Relationships']:
            if relationship['Type'] == 'CHILD':
                for child_id in relationship['Ids']:
                    word = blocks_map[child_id]
                    if word['BlockType'] == 'WORD':
                        text += word['Text'] + ' '
                    if word['BlockType'] == 'SELECTION_ELEMENT':
                        if word['SelectionStatus'] == 'SELECTED':
                            text += 'X '    

                                
    return text

def print_kvs(kvs):
    for key, value in kvs.items():
        print(key, ":", value)


def search_value(kvs, search_key):
    for key, value in kvs.items():
        if re.search(search_key, key, re.IGNORECASE):
            return value

def main_KV(file_name,fout):

    key_map, value_map, block_map = get_kv_map(file_name)

    # Get Key Value relationship
    kvs = get_kv_relationship(key_map, value_map, block_map)
    print("\n\n== FOUND KEY : VALUE pairs ===\n")
    fout.write(str(kvs))
    #print_kvs(kvs)

    # Start searching a key value
    ''' while input('\n Do you want to search a value for a key? (enter "n" for exit) ') != 'n':
        search_key = input('\n Enter a search key:')
        print('The value is:', search_value(kvs, search_key)) '''


## Main Function

In [100]:
root_dir = os.getcwd()
data_dir = os.path.join(root_dir,'data')
data_dir.split('\\')[-1]


'data'

In [101]:

if __name__ == '__main__':
    
    #filename
    root_dir = os.getcwd()
    data_dir = os.path.join(root_dir,'data')
    print(data_dir)
    save_dir =  os.path.join(root_dir,'Images')
    # convert_pdf_Img(data_dir,save_dir,res=400)
    
    textract_job(root_dir)

C:\Users\lenovo\PycharmProjects\aws-textract1\data
1
MAKE CHOICE FOR THE INPUT:
KV == KEY_VALUE PAIR 
TABLE == TABLE OUTPUT
enter the input : TABLE
Invoice_2_0.png
Image loaded C:\Users\lenovo\PycharmProjects\aws-textract1\Test_Inv\Invoice_2\Invoice_2_0.png
Table_1\Invoice_2_0.png
Table_2\Invoice_2_0.png
Table_3\Invoice_2_0.png
Invoice_2_1.png
Image loaded C:\Users\lenovo\PycharmProjects\aws-textract1\Test_Inv\Invoice_2\Invoice_2_1.png
Table_1\Invoice_2_1.png
Table_2\Invoice_2_1.png
Table_3\Invoice_2_1.png
Invoice_2_10.png
Image loaded C:\Users\lenovo\PycharmProjects\aws-textract1\Test_Inv\Invoice_2\Invoice_2_10.png
Table_1\Invoice_2_10.png
Table_2\Invoice_2_10.png
Table_3\Invoice_2_10.png
Invoice_2_11.png
Image loaded C:\Users\lenovo\PycharmProjects\aws-textract1\Test_Inv\Invoice_2\Invoice_2_11.png
Table_1\Invoice_2_11.png
Table_2\Invoice_2_11.png
Table_3\Invoice_2_11.png
Invoice_2_12.png
Image loaded C:\Users\lenovo\PycharmProjects\aws-textract1\Test_Inv\Invoice_2\Invoice_2_12.png
Ta

In [2]:
import pandas as pd

## converting csv into excel 

In [3]:
# csv_to_excel.py

import csv
import openpyxl


def csv_to_excel(csv_file, excel_file):
    csv_data = []
    with open(csv_file) as file_obj:
        reader = csv.reader(file_obj)
        for row in reader:
            csv_data.append(row)

    workbook = openpyxl.Workbook()
    sheet = workbook.active
    for row in csv_data:
        sheet.append(row)
    workbook.save(excel_file)


if __name__ == "__main__":
    csv_to_excel("output.csv", "books.xlsx")

In [1]:
df = pd.read_excel('books.xlsx')

NameError: name 'pd' is not defined

In [105]:
df.head(15)


,Table: Table_1\Invoice_2_0.png,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,(i) * Corporate Identification Number (CIN) of...,U01211TN1997PTC038419,Pre-fill,NaN,NaN,NaN,NaN,NaN,NaN
2,Global Location Number (GLN) of the company,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Permanent Account Number (PAN) of the company,AAACC4209Q,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,(ii) (a) Name of the company,CPF (INDIA) PRIVATELIMITED,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Preprocessing the table data

In [106]:
# removing only those rows which are having all NAN values 
df = df.dropna(how='all')
df.head(20)

,Table: Table_1\Invoice_2_0.png,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
1,(i) * Corporate Identification Number (CIN) of...,U01211TN1997PTC038419,Pre-fill,NaN,NaN,NaN,NaN,NaN,NaN
2,Global Location Number (GLN) of the company,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Permanent Account Number (PAN) of the company,AAACC4209Q,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,(ii) (a) Name of the company,CPF (INDIA) PRIVATELIMITED,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,Table: Table_2\Invoice_2_0.png,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,(v) Whether company is having share capital,X Yes,NaN,No,NaN,NaN,NaN,NaN,NaN
13,(vi) *Whether shares listed on recognized Stoc...,Yes,X,No,NaN,NaN,NaN,NaN,NaN
19,Table: Table_3\Invoice_2_0.png,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,Type of the Company,Category of the Company,Sub-category of the Company,NaN,NaN,NaN,NaN,NaN,NaN
22,Private Company,Company limited by shares,Indian Non-Government company,NaN,NaN,NaN,NaN,NaN,NaN


In [107]:
df.shape

(225, 9)

In [108]:
# reordering  the index values 
df.reset_index(drop=True, inplace=True)
df.head(10)

,Table: Table_1\Invoice_2_0.png,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,(i) * Corporate Identification Number (CIN) of...,U01211TN1997PTC038419,Pre-fill,NaN,NaN,NaN,NaN,NaN,NaN
1,Global Location Number (GLN) of the company,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Permanent Account Number (PAN) of the company,AAACC4209Q,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,(ii) (a) Name of the company,CPF (INDIA) PRIVATELIMITED,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Table: Table_2\Invoice_2_0.png,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,(v) Whether company is having share capital,X Yes,NaN,No,NaN,NaN,NaN,NaN,NaN
6,(vi) *Whether shares listed on recognized Stoc...,Yes,X,No,NaN,NaN,NaN,NaN,NaN
7,Table: Table_3\Invoice_2_0.png,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Type of the Company,Category of the Company,Sub-category of the Company,NaN,NaN,NaN,NaN,NaN,NaN
9,Private Company,Company limited by shares,Indian Non-Government company,NaN,NaN,NaN,NaN,NaN,NaN


In [109]:
# splitting the data frame based on a keyword 'TABLE:'
# grouped the data based on the keyword "Table: Table_1" using groupby function from pandas
df_group = df.groupby('Table: Table_1')
df_serial = df_group.get_group('Table: Table_1')

KeyError: 'Table: Table_1'

In [26]:
df_serial

,Table: Table_1,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
10,Table: Table_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,Table: Table_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43,Table: Table_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69,Table: Table_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84,Table: Table_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111,Table: Table_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
134,Table: Table_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
157,Table: Table_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
183,Table: Table_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
205,Table: Table_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
list_index = list(df_serial.index)
print(list_index)

[10, 21, 43, 69, 84, 111, 134, 157, 183, 205]


In [30]:
a = 0
b = list_index[0]
for i in list_index:
    b=i
    
    print(df[a:b-1])
    a=b-1

                                      Table: Table_1  \
0  (i) * Corporate Identification Number (CIN) of...   
1       Global Location Number (GLN) of the company    
2     Permanent Account Number (PAN) of the company    
3                      (ii) (a) Name of the company    
4                                     Table: Table_2   
5       (v) Whether company is having share capital    
6  (vi) *Whether shares listed on recognized Stoc...   
7                                     Table: Table_3   
8                               Type of the Company    

                    Unnamed: 1                    Unnamed: 2 Unnamed: 3  \
0       U01211TN1997PTC038419                      Pre-fill         NaN   
1                          NaN                           NaN        NaN   
2                  AAACC4209Q                            NaN        NaN   
3  CPF (INDIA) PRIVATELIMITED                            NaN        NaN   
4                          NaN                           NaN    